<a href="https://colab.research.google.com/github/mishraaayushman3/Fruits-Classifier/blob/master/Fruits_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import cv2
import os

In [0]:
!pip install kaggle
!mkdir -p ~/.kaggle

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!kaggle datasets download -d moltean/fruits

100% 706M/709M [00:08<00:00, 34.7MB/s]
100% 709M/709M [00:09<00:00, 82.4MB/s]


In [0]:
!unzip fruits.zip

In [0]:
directory1 = "fruits-360/Training/"
classes=["Apple Golden 1","Avocado","Banana","Cherry 1","Cocos","Kiwi","Lemon","Mango","Orange"]
train=[]
img_size=100
for fruit in classes:
    path = os.path.join(directory1,fruit)
    class_num = classes.index(fruit)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array,(img_size,img_size))
        train.append([img_array,class_num])

In [0]:
directory2 = "fruits-360/Test/"
classes=["Apple Golden 1","Avocado","Banana","Cherry 1","Cocos","Kiwi","Lemon","Mango","Orange"]
test=[]
img_size=100
for fruit in classes:
    path = os.path.join(directory2,fruit)
    class_num = classes.index(fruit)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array,(img_size,img_size))
        test.append([img_array,class_num])

In [0]:
import random

#training data
random.shuffle(train)

X_train=[]
Y_train=[]
for features,label in train:
    X_train.append(features)
    Y_train.append(label)
X_train = np.array(X_train)

#test data
random.shuffle(test)

X_test=[]
Y_test=[]
for features,label in test:
    X_test.append(features)
    Y_test.append(label)
X_test=np.array(X_test) 

In [0]:
X_train.shape

(4318, 100, 100)

In [0]:
X_train=X_train.reshape(-1,img_size,img_size,1)
X_train=X_train/255
X_test=X_test.reshape(-1,img_size,img_size,1)
X_test=X_test/255
print("shape of X_train= ",X_train.shape)
print("shape of X_test=  ",X_test.shape)

shape of X_train=  (4318, 100, 100, 1)
shape of X_test=   (1449, 100, 100, 1)


In [0]:
from keras.utils import to_categorical
Y_train=to_categorical(Y_train,num_classes=9)
Y_test=to_categorical(Y_test,num_classes=9)

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D

In [0]:
model=Sequential()
model.add(Conv2D(filters=8,kernel_size=(5,5),padding="Same",activation="relu",input_shape=(100,100,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=16,kernel_size=(4,4),padding="Same",activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=32,kernel_size=(4,4),padding="Same",activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(9,activation="softmax"))

W0728 11:57:56.583379 140160039729024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 11:57:56.631768 140160039729024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 11:57:56.640876 140160039729024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0728 11:57:56.693285 140160039729024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0728 11:57:56.696682 140160039729024 deprecation_wrapp

In [0]:
model.compile(optimizer = 'adam',loss = "categorical_crossentropy",metrics = ["accuracy"])

W0728 11:57:59.930432 140160039729024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0728 11:57:59.968991 140160039729024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
model.fit(X_train,Y_train,batch_size = 36,epochs = 5,validation_data = (X_test,Y_test))

W0728 11:58:11.656741 140160039729024 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 4318 samples, validate on 1449 samples
Epoch 1/5
4318/4318 [==============================] - 44s 10ms/step - loss: 0.7853 - acc: 0.7135 - val_loss: 0.1938 - val_acc: 0.9441
Epoch 2/5
4318/4318 [==============================] - 42s 10ms/step - loss: 0.0748 - acc: 0.9736 - val_loss: 0.1617 - val_acc: 0.9400
Epoch 3/5
4318/4318 [==============================] - 42s 10ms/step - loss: 0.0360 - acc: 0.9880 - val_loss: 0.1003 - val_acc: 0.9689
Epoch 4/5
4318/4318 [==============================] - 42s 10ms/step - loss: 0.0356 - acc: 0.9882 - val_loss: 0.1149 - val_acc: 0.9517
Epoch 5/5
4318/4318 [==============================] - 42s 10ms/step - loss: 0.0253 - acc: 0.9914 - val_loss: 0.0885 - val_acc: 0.9517


In [0]:
prediction = model.predict_classes(X_test)
print(prediction)

[6 0 8 ... 0 2 7]


In [0]:
mp = {}
mp[1] = """Per large apple, the nutrient profile looks like this :

Calories: 104 kcal
Carbohydrate: 27.6 g
Fiber: 2.8 g
Sugar: 21.8 g
Fat: 0.3 g
Protein: 0.6 g
Vitamin C: 14% RDA
Potassium: 6% RDA
Manganese: 4% RDA
Vitamin B6: 4% RDA
Vitamin B2: 4% RDA """

mp[2] = """Here is the nutrition breakdown per standard avocado :

Calories: 322 kcal
Carbohydrate: 17.1 g
Fiber: 13.5 g
Sugar: 0.2 g
Fat: 29.5 g
Protein: 4 g
Vitamin K: 53% RDA
Folate: 41% RDA
Vitamin C: 33% RDA
Potassium: 28% RDA
Vitamin B5: 28% RDA"""

mp[3] = """One medium banana has the following nutrient profile :

Calories: 105 kcal
Carbohydrate: 27.0 g
Fiber: 3.1 g
Sugar: 14.4 g
Fat: 0.4 g
Protein: 1.3 g
Vitamin B6: 22% RDA
Vitamin C: 17% RDA
Manganese: 16% RDA
Potassium: 12% RDA
Magnesium: 8% RDA"""
mp[4] = """Per 100g, here is the nutritional value of sour cherries:

Calories: 50 kcal
Carbohydrate: 12.2 g
 Fiber: 1.6 g
Sugar: 8.5 g
Fat: 0.3 g
Protein: 1.0 g
Vitamin A: 26% RDA
Vitamin C: 17% RDA
Manganese: 6% RDA
Potassium: 5% RDA
Copper: 5% RDA"""
mp[5] = """Serving Size: 1 Side has the follwing nutritional composition:
Calories: 70
Fat: 0g
Carbs: 18g
Protein: 1g"""
mp[6] = """Here are the nutritional details for one small kiwi (26);

Calories: 46.4 kcal
Carbohydrate: 11.1 g
Fiber: 2.3 g
Sugar: 6.8 g
Fat: 0.4 g
Protein: 0.9 g
Vitamin C: 117% RDA
Vitamin K: 38% RDA
Potassium: 7% RDA
Vitamin E: 6% RDA
Copper: 4% RDA
"""
mp[7] = """The fruit offers the following vitamins and minerals per standard-sized fruit :

Calories: 25.8 kcal
Carbohydrate: 25.4 g
Fiber: 1.6 g
Sugar: 1.5 g
Fat: 0.2 g
Protein: 0.6 g
Vitamin C: 51% RDA
Calcium: 2% RDA
Potassium: 2% RDA
Vitamin B1: 2% RDA
Vitamin B6: 2% RDA"""
mp[8] = """Per cup (165g) serving, mangoes provide :

Calories: 107 kcal
Carbohydrate: 28.1 g
Fiber: 3.0 g
Sugar: 24.4 g
Fat: 0.4 g
Protein: 0.8 g
Vitamin C: 76% RDA
Vitamin A: 25% RDA
Vitamin B6: 11% RDA
Copper: 9% RDA
Vitamin E: 9% RDA
"""
mp[9] = """One large orange supplies :

Calories: 86.5 kcal
Carbohydrate: 78.8 g
Fiber: 4.4 g
Sugar: 17.2 g
Fat: 0.2 g
Protein: 1.7 g
Vitamin C: 163% RDA
Folate: 14% RDA
Vitamin B1: 11% RDA
Potassium: 10% RDA
Vitamin A: 8% RDA
"""


In [0]:
img = cv2.imread("image.jpg",cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img,(img_size,img_size))
sample = [img]
sample = np.array(sample)
sample = np.reshape(sample,(-1,img_size,img_size,1))
fruit = model.predict_classes(sample)

print(mp[fruit[0]])

Per large apple, the nutrient profile looks like this :

Calories: 104 kcal
Carbohydrate: 27.6 g
Fiber: 2.8 g
Sugar: 21.8 g
Fat: 0.3 g
Protein: 0.6 g
Vitamin C: 14% RDA
Potassium: 6% RDA
Manganese: 4% RDA
Vitamin B6: 4% RDA
Vitamin B2: 4% RDA 
